# Using the Groq API in Python – A Complete Guide


## 1. Overview & Library Installation

**Theory:**
Groq offers a REST-based API that’s mostly compatible with the OpenAI Chat Completions interface. Fortunately, there’s an official Python client — `groq` — which wraps API calls in a clean, typed SDK.

**Installation:**

```bash
pip install groq
```


## 2. Basic Usage (Synchronous)

**Theory:** The `Groq` client simplifies authentication, request formatting, and response parsing.

In [ ]:

import os
from groq import Groq

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain quantum entanglement in simple terms."}
    ],
    model="llama3-8b-8192"
)

print(response.choices[0].message.content)


## 3. Asynchronous Usage with Streaming
**Theory:** If your workflow needs async operations (e.g., in web apps or event loops), use AsyncGroq.

In [ ]:

import os
import asyncio
from groq import AsyncGroq

async def main():
    client = AsyncGroq(api_key=os.environ.get("GROQ_API_KEY"))
    response = await client.chat.completions.create(
        messages=[{"role": "user", "content": "Tell me a joke."}],
        model="llama3-70b-8192",
        stream=True
    )
    async for chunk in response:
        print(chunk.choices[0].delta.content, end="")

asyncio.run(main())


This streams content in real time, printing as tokens arrive. 

## 4. Advanced: Accessing Raw or Streaming HTTP Responses
Sometimes you may want low-level control—inspect headers, stream manually, or handle undocumented features.

**Features:**

- .with_raw_response: Access HTTP headers or raw data.

- .with_streaming_response: Stream response content manually via context managers.

In [ ]:

from groq import Groq

client = Groq()
with client.chat.completions.with_streaming_response.create(
    messages=[{"role":"user","content":"Hello?"}],
    model="llama3-8b-8192"
) as resp:
    print("Response headers:", resp.headers)
    for line in resp.iter_lines():
        print(line)


Useful when you need resource tracking or custom streaming workflows.

## 5. Integration with Code Interpreter (E2B)
Want to build an app that generates code and runs it safely? Groq can pair with E2B Code Interpreter.

**Setup Steps:**

1. Install packages:

`pip install groq e2b-code-interpreter python-dotenv`

2. Set environment variables for API keys (GROQ_API_KEY, E2B_API_KEY).

In [ ]:

from e2b_code_interpreter import Sandbox
from groq import Groq
import os

client = Groq(api_key=os.environ.get('GROQ_API_KEY'))
sbx = Sandbox()

response = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[{"role":"system","content":"Write Python to sum a list."}, {"role":"user","content":"Create code to sum [1,2,3,4,5]."}]
)
code = response.choices[0].message.content.split("```python")[1].split("```")[0]

print("Generating code:\n", code)
print("Execution:\n", sbx.run_code(code).logs.stdout[0])


This dynamically generates and runs Python in a sandbox. 

## 6. Using Groq with LangChain

If you're already using LangChain, integrating Groq is smooth thanks to shared API semantics.

In [ ]:

import os
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

os.environ["GROQ_API_KEY"] = "your-key"
ai = ChatGroq(model="llama3-8b-8192", temperature=0.7, max_tokens=512)

prompt = PromptTemplate(input_variables=["task"], template="Write a Python program to {task}.")
chain = LLMChain(llm=ai, prompt=prompt)
print(chain.run(task="calculate factorial"))


Leverage existing LangChain patterns with Groq backend.


## Summary Table

| Task                      | Example Usage                                     |
| ------------------------- | ------------------------------------------------- |
| Install library           | `pip install groq`                                |
| Sync Chat Completion      | `Groq(...).chat.completions.create(…)`            |
| Async + Streaming         | `AsyncGroq(...)` + `stream=True`                  |
| Raw/Streaming HTTP access | `.with_raw_response` / `.with_streaming_response` |
| Code generation + sandbox | Groq + E2B code interpreter                       |
| LangChain integration     | `langchain_groq.ChatGroq`                         |
